In [1]:
import requests
import pandas as pd
import json
from io import StringIO

# Day 1

In [2]:
cookies = {
    "session": "53616c7465645f5ff7db31f30dbc39e8efa073d984f1019c423857460dd4104435d33265120f00431aafacd6a212e84f; _ga=GA1.2.1167308080.1638436952; _gid=GA1.2.1150076090.1638436952"
}
resp = requests.get("https://adventofcode.com/2021/day/1/input", cookies=cookies)
df = pd.read_csv(StringIO(resp.content.decode('utf-8')), sep = " ", names=["first"])

In [4]:
def count_increased(df):
    df = df.copy()
    df['second'] = df['first'].shift(-1)
    return (df['second'] > df['first']).sum()

In [5]:
count_increased(df)

1529

In [6]:
def count_increased_triplets(df):
    df = df.copy()
    df = df.rolling(window=3).sum()
    return count_increased(df)

In [7]:
count_increased_triplets(df)

1567

# Day 2

In [8]:
cookies = {
    "session": "53616c7465645f5ff7db31f30dbc39e8efa073d984f1019c423857460dd4104435d33265120f00431aafacd6a212e84f; _ga=GA1.2.1167308080.1638436952; _gid=GA1.2.1150076090.1638436952"
}
resp = requests.get("https://adventofcode.com/2021/day/2/input", cookies=cookies)
df = pd.read_csv(StringIO(resp.content.decode('utf-8')), sep = " ", names=["move", "value"])

In [9]:
def calculate_square_move_position(df):
    df = df.copy()
    agg_df = df.groupby("move").sum()
    fwd = agg_df.loc["forward", "value"]
    depth = agg_df.loc["down", "value"] - agg_df.loc["up", "value"]
    return fwd * depth

In [10]:
calculate_square_move_position(df)

1762050

In [11]:
def calculate_aim_move_position(df):
    df.copy()
    df = df.set_index("move")
    df['aim_diff'] = 0
    df['depth_diff'] = 0
    df['fwd_diff'] = 0
    df.loc["down", "aim_diff"] = df.loc["down", "value"]
    df.loc["up", "aim_diff"] = -df.loc["up", "value"]
    df['aim'] = df.aim_diff.cumsum()
    df.loc["forward", "fwd_diff"] = df.loc["forward"].value
    df.loc["forward", "depth_diff"] = df.loc["forward"].value * df.loc["forward"].aim
    return df['depth_diff'].sum() * df['fwd_diff'].sum()


In [12]:
calculate_aim_move_position(df)

1855892637

# Day 3

In [13]:
cookies = {
    "session": "53616c7465645f5ff7db31f30dbc39e8efa073d984f1019c423857460dd4104435d33265120f00431aafacd6a212e84f; _ga=GA1.2.1167308080.1638436952; _gid=GA1.2.1150076090.1638436952"
}
resp = requests.get("https://adventofcode.com/2021/day/3/input", cookies=cookies)
df = pd.read_csv(StringIO(resp.content.decode('utf-8')), sep = " ", names=["diag"], dtype=str)

In [14]:
def power_consumption(df):
    df = df.copy()
    bit_len = len(df.iloc[0, 0])
    df = df.diag.str.split("", expand = True).iloc[:, 1:-1]
    gamma_rate = int(df.mode().apply(lambda row: "".join(row.astype(str)), axis=1)[0], 2)
    epsilon_rate = 2**bit_len - 1 - gamma_rate
    return gamma_rate * epsilon_rate

In [15]:
power_consumption(df)

1092896

In [16]:
def bit_criteria(df, inversed=False):
    default = '0' if inversed else '1'
    new_df = df.copy()
    for i in new_df.columns:
        if new_df.shape[0] == 1:
            break
        counts = new_df.loc[:, i].value_counts(ascending=inversed)
        most_common = default if counts[0] == counts[1] else counts.index[0]
        new_df = new_df[new_df.loc[:, i] == most_common]
    return int(new_df.apply(lambda row: "".join(row.astype(str)), axis=1).iloc[0], 2)

In [17]:
def life_support(df):
    df = df.copy()
    df = df.diag.str.split("", expand = True).iloc[:, 1:-1].astype("category")
    oxygen = bit_criteria(df, False)
    co2 = bit_criteria(df, True)
    return oxygen * co2

In [18]:
life_support(df)

4672151

# Day 4